In [ ]:
import pandas as pd
import numpy as np
import ast
import random
import os
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

#### X test df

In [ ]:
df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = df.fillna(-1)
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_test_df = pd.concat([X_test, y_test], axis=1)
X_test_df = X_test_df.reset_index(drop=True)
X_test_df.head()

#### The test dfs

In [ ]:
dqn_test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
dqn_test_df.head()

In [ ]:
dqn_test_df[dqn_test_df.y_actual==dqn_test_df.y_pred].y_pred.value_counts()

In [ ]:
dt_test_df = pd.read_csv('../../test_dfs/many_features/dt_test_df_0.1.csv')
dt_test_df.head()

In [ ]:
test_df = dqn_test_df.copy()
test_df = test_df.rename({'y_pred': 'dqn_y_pred'}, axis=1)
test_df['dt_y_pred'] = dt_test_df['y_pred']
test_df.head()

In [ ]:
test_df.y_actual.value_counts()

In [ ]:
test_df[test_df.y_actual==test_df.dqn_y_pred].dt_y_pred.value_counts()

#### Episodes where DQN performs better than DT agent

In [ ]:
better_dqn_df = test_df[(test_df.y_actual==test_df.dqn_y_pred) & (test_df.y_actual!=test_df.dt_y_pred)]
better_dqn_df.head()

In [ ]:
better_dqn_df.dt_y_pred.value_counts()

In [ ]:
better_dqn_df.dqn_y_pred.value_counts()

In [ ]:
better_dqn_df[better_dqn_df.y_actual==5].trajectory.value_counts()

#### Episodes where DT agent is better than DQN

In [ ]:
better_dt_df = test_df[(test_df.y_actual!=test_df.dqn_y_pred) & (test_df.y_actual==test_df.dt_y_pred)]
better_dt_df.head()

In [ ]:
len(better_dt_df)

In [ ]:
better_dt_df.dt_y_pred.value_counts()

In [ ]:
better_dqn_df.dqn_y_pred.value_counts()

In [ ]:
test_df[(test_df.y_actual==2) & (test_df.dqn_y_pred!=2)].dt_y_pred.value_counts()

#### Shorter DQN paths

In [ ]:
shorters_df = test_df.copy()
shorters_df = shorters_df.rename({'trajectory':'dqn_trajectory', 'episode_length':'dqn_length'}, axis=1)
shorters_df['dt_trajectory'] = dt_test_df['trajectory']
shorters_df['dt_length'] = dt_test_df['episode_length']
shorters_df = shorters_df.drop(['index', 'reward'], axis=1)
shorters_df['dt_trajectory'] = shorters_df['dt_trajectory'].apply(lambda x: ast.literal_eval(x))
shorters_df['dqn_trajectory'] = shorters_df['dqn_trajectory'].apply(lambda x: ast.literal_eval(x))
shorters_df.head()

In [ ]:
shorters_dqn_df = pd.DataFrame()
for i, row in shorters_df.iterrows():
    if len(row.dqn_trajectory) < len(row.dt_trajectory):
        if row.dqn_y_pred == row.y_actual:
            shorters_dqn_df = shorters_dqn_df.append(row)
shorters_dqn_df.head()

In [ ]:
assert shorters_dqn_df.y_actual.tolist() == shorters_dqn_df.dqn_y_pred.tolist()

In [ ]:
shorters_dqn_df.dqn_y_pred.value_counts()

In [ ]:
shorter_dqn_df_1 = shorters_dqn_df[shorters_dqn_df.dqn_y_pred==4]
shorter_dqn_df_1.head()

In [ ]:
shorter_dqn_df_1.dqn_length.unique()

In [ ]:
X_test_df.iloc[shorter_dqn_df_1.index].describe()

In [ ]:
X_test_df.iloc[17]

In [ ]:
shorter_dqn_df_1.tail()

In [ ]:
shorter_dqn_df_1[shorter_dqn_df_1.dqn_length==5]

In [ ]:
shorter_dqn_df_1.dqn_length.unique()

In [ ]:
# shorter_dqn_df_1[shorter_dqn_df_1.dqn_length==4].dt_length.unique()

In [ ]:
X_test_df.iloc[shorter_dqn_df_1.index].describe()

In [ ]:
analytics_test_df =shorter_dqn_df_1[shorter_dqn_df_1.dqn_length==3]
analytics_test_df.head()

In [ ]:
index_list = list(analytics_test_df.index)
analytics_x_test_df = X_test_df.iloc[index_list]
analytics_x_test_df.describe()

In [ ]:
shorter_dqn_df_1.dqn_trajectory.value_counts()